In [ ]:

# %pip install pandas yfinance requests beautifulsoup4 lxml plotly


In [1]:

import re, time, random, requests, pandas as pd
from datetime import datetime
import yfinance as yf
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio

pd.set_option("display.max_rows", 8)
pd.set_option("display.width", 120)

# ----------------- HTTP helpers -----------------
def polite_session(ua=None, delay=(0.7, 1.4)):
    s = requests.Session()
    s.headers.update({
        "User-Agent": ua or ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                             "AppleWebKit/537.36 (KHTML, like Gecko) "
                             "Chrome/124.0.0.0 Safari/537.36"),
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com/",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
    })
    s._delay = delay
    return s

def _pause(session):
    lo, hi = session._delay
    time.sleep(random.uniform(lo, hi))

def _to_number(s):
    s = str(s).strip().replace(",", "")
    mult = 1.0
    if s.endswith(("B","b")):
        mult = 1_000_000_000.0
        s = s[:-1]
    elif s.endswith(("M","m")):
        mult = 1_000_000.0
        s = s[:-1]
    try:
        return float(s) * mult
    except:
        return None

def _df_clean(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [str(c).strip().title() for c in df.columns]
    if "Date" not in df.columns:
        df.rename(columns={df.columns[0]:"Date"}, inplace=True)
    rev = next((c for c in df.columns if "revenue" in c.lower()), None) or df.columns[1]
    df = df[["Date", rev]].rename(columns={rev:"Revenue"})
    df["Revenue"] = (df["Revenue"].astype(str)
                     .str.replace("$","",regex=False)
                     .str.replace("USD","",regex=False)
                     .str.strip())
    df["Revenue"] = df["Revenue"].apply(_to_number)
    dt_iso = pd.to_datetime(df["Date"], errors="coerce")
    dt_us  = pd.to_datetime(df["Date"], format="%b %d, %Y", errors="coerce")
    df["Date"] = dt_iso.fillna(dt_us)
    df = df.dropna(subset=["Date","Revenue"]).sort_values("Date").reset_index(drop=True)
    return df

def _parse_text_block(text: str) -> pd.DataFrame:
    sec = re.search(r"Quarterly Revenue(.*?)(Annual Revenue|$)", text, flags=re.S|re.I)
    block = sec.group(1) if sec else text
    rows = []
    rows += re.findall(r"(\d{4}-\d{2}-\d{2})\s+([\d,\.]+[MBmb]?)", block)
    rows += re.findall(r"([A-Z][a-z]{2}\s+\d{1,2},\s+\d{4})\s+([\d,\.]+[MBmb]?)", block)
    for q, v in re.findall(r"(Q[1-4]\s+\d{4})\s+([\d,\.]+[MBmb]?)", block, flags=re.I):
        year = int(re.search(r"\d{4}", q).group())
        qn = int(re.search(r"Q([1-4])", q, flags=re.I).group(1))
        month_day = {1:(3,31), 2:(6,30), 3:(9,30), 4:(12,31)}[qn]
        rows.append((f"{year:04d}-{month_day[0]:02d}-{month_day[1]:02d}", v))
    if not rows:
        raise ValueError("Sin filas reconocibles en texto.")
    df = pd.DataFrame(rows, columns=["Date","Revenue"])
    df["Revenue"] = df["Revenue"].apply(_to_number)
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.dropna(subset=["Date","Revenue"]).sort_values("Date").reset_index(drop=True)
    return df

# ----------------- Revenue sources -----------------
def revenue_from_macrotrends_html(ticker: str, slug: str) -> pd.DataFrame:
    url = f"https://www.macrotrends.net/stocks/charts/{ticker}/{slug}/revenue"
    s = polite_session()
    _ = s.get("https://www.macrotrends.net/", timeout=20); _pause(s)
    r = s.get(url, timeout=20)
    r.raise_for_status()
    tables = pd.read_html(r.text)
    for t in tables:
        try:
            df = _df_clean(t)
            if len(df) >= 8 and {"Date","Revenue"}.issubset(df.columns):
                return df
        except Exception:
            pass
    raise ValueError("No se hallaron tablas útiles en HTML.")

def revenue_from_text_mirror(ticker: str, slug: str) -> pd.DataFrame:
    url = f"https://r.jina.ai/http://www.macrotrends.net/stocks/charts/{ticker}/{slug}/revenue"
    s = polite_session()
    r = s.get(url, timeout=20)
    r.raise_for_status()
    return _parse_text_block(r.text)

def revenue_from_yfinance(symbol: str) -> pd.DataFrame:
    t = yf.Ticker(symbol)
    q = t.quarterly_income_stmt
    if q is None or q.empty:
        q = t.quarterly_financials
    if q is None or q.empty:
        return pd.DataFrame(columns=["Date","Revenue"])
    df = q.T.reset_index().rename(columns={"index":"Date"})
    revenue_col = None
    for c in df.columns:
        cl = str(c).lower()
        if "revenue" in cl or "sales" in cl:
            revenue_col = c; break
    if revenue_col is None:
        for c in df.columns:
            if "rev" in str(c).lower():
                revenue_col = c; break
    if revenue_col is None:
        return pd.DataFrame(columns=["Date","Revenue"])
    out = df[["Date", revenue_col]].rename(columns={revenue_col:"Revenue"})
    out["Date"] = pd.to_datetime(out["Date"], errors="coerce")
    out["Revenue"] = pd.to_numeric(out["Revenue"], errors="coerce")
    out = out.dropna(subset=["Date","Revenue"]).sort_values("Date").reset_index(drop=True)
    return out

def get_quarterly_revenue(ticker: str, slug: str) -> pd.DataFrame:
    for fn, label in [(revenue_from_macrotrends_html, "Macrotrends (HTML)"),
                      (revenue_from_text_mirror, "Macrotrends (texto espejo)"),
                      (revenue_from_yfinance, "yfinance (quarterly)")]:
        try:
            df = fn(ticker, slug) if fn!=revenue_from_yfinance else fn(ticker)
            print("Fuente QRev:", label, f"→ filas: {len(df)}")
            return df
        except Exception as e:
            print("Aviso:", label, "falló →", e)
    return pd.DataFrame(columns=["Date","Revenue"])

# ----------------- Stock & plotting -----------------
def get_stock_data(ticker: str) -> pd.DataFrame:
    t = yf.Ticker(ticker)
    df = t.history(period="max").reset_index()
    keep = [c for c in ["Date","Open","High","Low","Close","Volume","Dividends","Stock Splits"] if c in df.columns]
    return df[keep]

def _normalize_dates(df, date_col="Date"):
    if df is None or df.empty or date_col not in df.columns:
        return df
    out = df.copy()
    out[date_col] = pd.to_datetime(out[date_col], errors="coerce", utc=True).dt.tz_convert(None)
    return out.dropna(subset=[date_col]).sort_values(date_col).reset_index(drop=True)

def _normalize_rev_df(df: pd.DataFrame) -> pd.DataFrame:
    if df is None or df.empty:
        return pd.DataFrame(columns=["Date","Revenue"])
    out = df.copy()
    cols = {c.lower(): c for c in out.columns}
    date_col   = cols.get("date", next(iter(out.columns)))
    revenue_col = None
    for c in out.columns:
        if "rev" in str(c).lower():
            revenue_col = c; break
    if revenue_col is None and len(out.columns)>1:
        revenue_col = out.columns[1]
    out = out[[date_col, revenue_col]].rename(columns={date_col:"Date", revenue_col:"Revenue"})
    out["Date"] = pd.to_datetime(out["Date"], errors="coerce", utc=True).dt.tz_convert(None)
    out["Revenue"] = pd.to_numeric(out["Revenue"], errors="coerce")
    out = out.dropna(subset=["Date","Revenue"]).sort_values("Date").reset_index(drop=True)
    return out

def ensure_quarterly_revenue(symbol: str, current_rev: pd.DataFrame, stock_df: pd.DataFrame) -> pd.DataFrame:
    rev = _normalize_rev_df(current_rev)
    if not rev.empty:
        print(f"[OK] Usando revenue existente de {symbol}: {len(rev)} filas")
        return rev
    rev = revenue_from_yfinance(symbol)
    rev = _normalize_rev_df(rev)
    if not rev.empty:
        print(f"[OK] Usando revenue de yfinance para {symbol}: {len(rev)} filas")
        return rev
    print(f"[AVISO] Sin revenue real para {symbol}. Se usará un PROXY trimestral de Close.")
    s = stock_df.dropna(subset=["Date","Close"]).copy()
    s = s.set_index("Date").sort_index()
    proxy = s["Close"].resample("Q").last().dropna().reset_index().rename(columns={"Close":"Revenue"})
    return _normalize_rev_df(proxy)

def make_graph_safe(stock_data: pd.DataFrame, revenue_data: pd.DataFrame, stock_name: str):
    # normalizar fechas
    stock_data = _normalize_dates(stock_data, "Date")
    revenue_ok = _normalize_rev_df(revenue_data)
    use_proxy = False
    if revenue_ok.empty:
        use_proxy = True
        s = stock_data.dropna(subset=["Date","Close"]).set_index("Date").sort_index()
        revenue_ok = s["Close"].resample("Q").last().dropna().reset_index().rename(columns={"Close":"Revenue"})
        revenue_ok = _normalize_rev_df(revenue_ok)

    max_rev_date = revenue_ok["Date"].max()
    stock_plot = stock_data[stock_data["Date"] <= max_rev_date].copy()

    fig = make_subplots(rows=1, cols=2, subplot_titles=(f"{stock_name} Historical Share Price",
                                                        f"{stock_name} Quarterly Revenue" + (" (proxy)" if use_proxy else "")),
                        specs=[[{"type":"scatter"}, {"type":"bar"}]], horizontal_spacing=0.12)
    fig.add_trace(go.Scatter(x=stock_plot["Date"], y=stock_plot["Close"], name="Close", mode="lines"), row=1, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_yaxes(title_text="Close (USD)", row=1, col=1)

    fig.add_trace(go.Bar(x=revenue_ok["Date"], y=revenue_ok["Revenue"], name="Revenue"), row=1, col=2)
    fig.update_xaxes(title_text="Date", row=1, col=2)
    fig.update_yaxes(title_text=("Revenue (USD)" if not use_proxy else "Revenue proxy"), row=1, col=2)

    fig.update_layout(title_text=f"{stock_name}: Price vs Quarterly Revenue", showlegend=False, height=520, width=1000)
    return fig


## Q1 — Tesla stock (yfinance): reset index, save y head( )

In [2]:

tesla_data = get_stock_data("TSLA")
tesla_data = _normalize_dates(tesla_data, "Date")
tesla_data.to_csv("tesla_data.csv", index=False)
tesla_data.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 04:00:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 04:00:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 04:00:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 04:00:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 04:00:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


## Q2 — Tesla revenue (webscraping): **Tesla Quarterly Revenue** y tail( )

In [3]:

tesla_revenue = get_quarterly_revenue("TSLA", "tesla")
tesla_revenue = ensure_quarterly_revenue("TSLA", tesla_revenue, tesla_data)
tesla_revenue.to_csv("tesla_quarterly_revenue.csv", index=False)
tesla_revenue.tail()


Aviso: Macrotrends (HTML) falló → 403 Client Error: Forbidden for url: https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue
Aviso: Macrotrends (texto espejo) falló → 451 Client Error: Unavailable For Legal Reasons for url: https://r.jina.ai/http://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue
Fuente QRev: yfinance (quarterly) → filas: 5
[OK] Usando revenue existente de TSLA: 5 filas


,Date,Revenue
0,2024-06-30,2.092200e+10
1,2024-09-30,2.018500e+10
2,2024-12-31,2.152800e+10
3,2025-03-31,1.618200e+10
4,2025-06-30,1.861800e+10


## Q3 — GameStop stock (yfinance): reset index, save y head( )

In [4]:

gme_data = get_stock_data("GME")
gme_data = _normalize_dates(gme_data, "Date")
gme_data.to_csv("gme_data.csv", index=False)
gme_data.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 05:00:00,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 05:00:00,1.712708,1.716074,1.670626,1.683251,11021600,0.0,0.0
2,2002-02-15 05:00:00,1.683251,1.687459,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 05:00:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 05:00:00,1.615920,1.662209,1.603295,1.662209,6892800,0.0,0.0


## Q4 — GameStop revenue (webscraping): **GameStop Quarterly Revenue** y tail( )

In [5]:

gme_revenue = get_quarterly_revenue("GME", "gamestop")
gme_revenue = ensure_quarterly_revenue("GME", gme_revenue, gme_data)
gme_revenue.to_csv("gme_quarterly_revenue.csv", index=False)
gme_revenue.tail()


Aviso: Macrotrends (HTML) falló → 403 Client Error: Forbidden for url: https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue
Aviso: Macrotrends (texto espejo) falló → 451 Client Error: Unavailable For Legal Reasons for url: https://r.jina.ai/http://www.macrotrends.net/stocks/charts/GME/gamestop/revenue
Fuente QRev: yfinance (quarterly) → filas: 5
[OK] Usando revenue existente de GME: 5 filas


,Date,Revenue
0,2024-07-31,549500000.0
1,2024-10-31,603100000.0
2,2025-01-31,919200000.0
3,2025-04-30,479600000.0
4,2025-07-31,689100000.0


## Q5 — Gráfico Tesla (make_graph_safe)

In [6]:

fig_tsla = make_graph_safe(tesla_data, tesla_revenue, "Tesla")
pio.write_html(fig_tsla, file="tsla_dashboard.html", auto_open=False)
"Guardado: tsla_dashboard.html"


'Guardado: tsla_dashboard.html'

## Q6 — Gráfico GameStop (make_graph_safe)

In [7]:

fig_gme = make_graph_safe(gme_data, gme_revenue, "GameStop")
pio.write_html(fig_gme, file="gme_dashboard.html", auto_open=False)
"Guardado: gme_dashboard.html"


'Guardado: gme_dashboard.html'